In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import parameters

dropbox = '/Users/jameswhittington/Dropbox/gen_struct_know/Summaries/'
this_comp = '/Users/jameswhittington/Documents/DPHIL/Saved models/TEM/this_comp/'
gcl_path = '/Users/jameswhittington/Documents/DPHIL/Saved models/TEM/gcl_1/'
bmrc_path = '/Users/jameswhittington/Documents/DPHIL/Saved models/TEM/bmrc/'
saved_path = '/Users/jameswhittington/Documents/DPHIL/Saved models/Saved_TEM/'
failed_path = '/Users/jameswhittington/Documents/DPHIL/Saved models/Failed_TEM/'

save_dirs = bmrc_path
date = '2022-06-27'
run = 3

params_1, saved_dir_1 = parameters.get_params(save_dirs, date, run)

save_dirs = dropbox
date = '2022-08-18'
run = 0

params_2, saved_dir_2 = parameters.get_params(save_dirs, date, run, not_this_dir=saved_dir_1)

compared = parameters.compare_params(params_1, params_2)
for message in sorted(compared):
    print(message)

In [ ]:
import os
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def get_all_params(bmrc_path):
    
    date_paths = [x for x in os.listdir(bmrc_path) if 'DS_Store' not in x]
    runs, dates = [], []
    for date_path in date_paths:
        dates_ = [date_path for x in os.listdir(bmrc_path + date_path) if 'DS_Store' not in x]
        dates.extend(dates_)

        runs_ = [x[3:] for x in os.listdir(bmrc_path + date_path) if 'DS_Store' not in x] 
        runs.extend(runs_)

    # load params
    params_ = []
    for date, run in zip(dates, runs):
        params_1, saved_dir_1 = parameters.get_params(bmrc_path, date, run, print_where=False)
        params_.append(flatten(params_1))
    
    return params_

params_good = get_all_params(bmrc_path)
params_bad = get_all_params(failed_path)

# now go through all params and pull out which things are similar and which are different
# could do all pairwise compare params...

# get all keys:
keys = []
for param in params_good + params_bad:
    keys.extend(param.keys())
keys = sorted(list(set(keys)))

def add_param(dict_, params_, key):
    for param in params_:
        try:
            dict_[key]
        except KeyError:
            dict_[key] = []
        try: 
            if isinstance(param[key], list):
                try:
                    dict_[key].append(np.mean(param[key]))
                except TypeError:
                    dict_[key].append(param[key])
            else:
                dict_[key].append(param[key])
            
        except KeyError:
            continue
    return dict_

dict_good = {}
dict_bad = {}
for key in keys:
    dict_good = add_param(dict_good, params_good, key)
    dict_bad = add_param(dict_bad, params_bad, key)
    
# now compare good and bad:
diffs_g, diffs_b = {}, {}
for i, key in enumerate(dict_good.keys()):
    try:
        if len(dict_good[key]) == 0 or len(dict_bad[key]) == 0:
            continue
    except KeyError:
        continue
        
    all_key = dict_good[key] + dict_bad[key]
    try:
        if len([list(x) for x in set(tuple(x) for x in all_key)]) <= 1:
            continue
    except TypeError:
        if len(np.unique(all_key)) <= 1:
            continue

    diffs_g[key] = dict_good[key]
    diffs_b[key] = dict_bad[key]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,16))
n = int(np.ceil(np.sqrt(len(diffs_b.keys()))))
for i, key in enumerate(diffs_g.keys()):
    plt.subplot(n,n,i+1)
    ax = plt.gca()
    
    ax.scatter([str(x) for x in diffs_g[key]], [0+np.random.randn()*0.1 for _ in diffs_g[key]], label='good')
    ax.scatter([str(x) for x in diffs_b[key]], [1+np.random.randn()*0.1 for _ in diffs_b[key]], label='bad')
    
    plt.title(key, color='yellow')    
    ax.xaxis.label.set_color('yellow')
    ax.tick_params(axis='x', colors='yellow')
    ax.yaxis.label.set_color('yellow')
    ax.tick_params(axis='y', colors='yellow')

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
import os
[x[0] for x in os.walk('../Summaries/') if 'save' in x[0] and 'run' in x[0] and 'iter' not in x[0]]

In [ ]:
save_dirs = [bmrc_path, saved_path]

# search through all saved models looking for something
keys = ['softmax_beta'] 
vals_desired = []

parameters.find_model_with_params(save_dirs, keys, vals_desired)

In [ ]:
# remove most stuff from failed runs
import shutil
import os

failed_path = '/Users/jameswhittington/Documents/DPHIL/Saved models/Failed_TEM/'
dirs = [x for x in next(os.walk(failed_path))[1]]

for dir_ in dirs:
    directory = failed_path + dir_
    run_dir = [x for x in next(os.walk(directory))[1] if 'run' in x]
    for run in run_dir:
        path = directory + '/' + run

        # remove processed_runs
        save_path = path + '/save'
        dirs_save = [save_path + '/' + x for x in next(os.walk(save_path))[1] if 'params' not in x]
        for dirpath in dirs_save:
            if os.path.exists(dirpath) and os.path.isdir(dirpath):
                shutil.rmtree(dirpath)
                
        # remove all but last model
        model_path = path + '/model'
        poss_files = [int(x.split('.index')[0].split('tem_')[1]) for x in next(os.walk(model_path))[2] if 'index' in x]
        if len(poss_files) == 0:
            continue
        max_index = max(poss_files)
        rem_model_paths = [model_path + '/' + x for x in next(os.walk(model_path))[2] if 'checkpoint' not in x and str(max_index) not in x]
        for dirpath in rem_model_paths:
            os.remove(dirpath)

In [ ]:
# DELETING OLD SAVED FILES - ONLY AFTER SAVED ON EXTERNAL DRIVE + KEEP PARAMS FILE

import shutil
import os

save_dir = '/Users/jameswhittington/Documents/DPHIL/Saved models/saved summaries/'
old_dirs = [x for x in next(os.walk(save_dir))[1] if '2018' in x or '2019' in x or '2020' in x]
for old_dir in old_dirs:
    # delete all but params
    directory = save_dir + old_dir
    
    # OLD STYLE FOLDER SETUP   
    dirpaths = [directory + '/model', directory + '/train']
    for dirpath in dirpaths:
        if os.path.exists(dirpath) and os.path.isdir(dirpath):
            print(dirpath)
            shutil.rmtree(dirpath)
    """ 
    dirpath = directory + '/save'
    if os.path.exists(dirpath) and os.path.isdir(dirpath):        
        try:
            run_dir = [x for x in next(os.walk(dirpath))[1] if 'run' in x]
            for run in run_dir:
                path = dirpath + '/' + run
                list_of_files = os.listdir(path)
                for file in list_of_files:
                    if 'par' not in file:
                        os.remove(path + '/' + file)

        except:
            pass
    """
    # NEW STYLE FOLDER SET UP
    run_dir = [x for x in next(os.walk(directory))[1] if 'run' in x]
    for run in run_dir:
        path = directory + '/' + run
        
        dirpaths = [path + '/model', path + '/train']
        for dirpath in dirpaths:
            if os.path.exists(dirpath) and os.path.isdir(dirpath):
                shutil.rmtree(dirpath)
        """ 
        dirpath = path + '/save'
        if os.path.exists(dirpath) and os.path.isdir(dirpath):      
            list_of_files = os.listdir(dirpath)
            for file in list_of_files:
                if 'par' not in file:
                    #print(dirpath + '/' + file)
                    os.remove(dirpath + '/' + file)
        """
